In [1]:
using Pkg

Pkg.activate("..")
using Revise

using PeriDyn
using PDMesh

using Random
using Distributed, Folds


  Activating environment at `/mnt/D/git_repos/PeriDyn/Project.toml`
┌ Info: Precompiling PeriDyn [c3db0ce0-6d7e-41b5-be11-bcb6aafada44]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **



In [2]:
PeriDyn.TIMEIT_REF[] = true
Random.seed!(42)

resolution = 1


1

In [38]:
x1, v1, y1, vol1, type1 = create(move(Cuboid([-2 2; -2 2; 0 20]), by=[0.0, 10.0, 0]), resolution=resolution, type=1)

v1[1,:] .= 0.0

hor1 = 3.0
mat_gen1 = GeneralMaterial(y1, v1, x1, vol1, type1, hor1, max_neigh=200)

K, G = 1.0, 1.0
den1 = 10000.0
cstretch1 = 0.5
mat_spec1 = OrdinaryStateBasedSpecific([K], [G], [cstretch1], [den1])
# mat_spec1 = SkipSpecific([den1])

block1 = PeridynamicsMaterial(mat_gen1, mat_spec1; name="block 1")


Average family members: 47.475


OrdinaryStateBasedMaterial("block 1", 1:1, GeneralMaterial
type: [1]
horizon: 3.0
particle size: [1.0]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [10000.0]
)

In [39]:
x2, v2, y2, vol2, type2 = create(move(Cuboid([-2 2; -2 2; 0 20]), by=[0.0, -10.0, 0]), resolution=resolution, type=2)

hor2 = 3.0
mat_gen2 = GeneralMaterial(y2, v2, x2, vol2, type2, hor2, max_neigh=200)

K2, G2 = 1.0, 1.0
den2 = 10000.0
cstretch2 = 0.5
mat_spec2 = OrdinaryStateBasedSpecific([K2], [G2], [cstretch2], [den2])
# mat_spec2 = SkipSpecific([den2])

block2 = PeridynamicsMaterial(mat_gen2, mat_spec2; name="block 2")

Average family members: 47.475


OrdinaryStateBasedMaterial("block 2", 2:2, GeneralMaterial
type: [2]
horizon: 3.0
particle size: [1.0]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [10000.0]
)

In [40]:
function delete(out, mask::BitArray)
    return out[1][:, mask], out[2][:, mask], out[3][:, mask], out[4][mask], out[5][mask]
end

"""
    delete!(obj::T, f::Function) where T

Delete mesh particle for object using boolean array from function f.
"""
function delete(obj::T, f::Function) where T  
    # x, v, y, vol, type
    function func(out)
        mask = vec(f(out)) .== false
        out = delete(out, mask) 
        return out
    end
    if isa(obj, Shape)
        return PostOpObj(obj, func)
    elseif isa(obj, PostOpObj)
        push!(obj.operations, func)       
        return obj         
    else
        error("Not allowed.")
    end
end

delete

In [41]:
res = 0.5*resolution
y2, v2, x2, vol2, type2 = create(delete(move(Cylinder(9.0, 2.0, 20), by=[0.0, 0.0, 0.0]), (out)->out[1][1,:] .< 3), resolution=res, type=3)

hor2 = 3.0*res
mat_gen2 = GeneralMaterial(y2, v2, x2, vol2, type2, hor2, max_neigh=200)

K2, G2 = 1.0, 1.0
den2 = 10000.0
cstretch2 = 0.5
mat_spec2 = OrdinaryStateBasedSpecific([K2], [G2], [cstretch2], [den2])
# mat_spec2 = SkipSpecific([den2])

block3 = PeridynamicsMaterial(mat_gen2, mat_spec2; name="cylinder")

Average family members: 74.38217821782179


OrdinaryStateBasedMaterial("cylinder", 3:3, GeneralMaterial
type: [3]
horizon: 1.5
particle size: [0.5003430521955505]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [10000.0]
)

In [56]:
x2, v2, y2, vol2, type2 = create(move(Cuboid([-2 2; -2 2; 0 20]), by=[14.0, 0.0, 0.0]), resolution=resolution, type=4)

v2[1,:] .= 0.01

hor2 = 3.0
mat_gen2 = GeneralMaterial(y2, v2, x2, vol2, type2, hor2, max_neigh=200)

K2, G2 = 1.0, 1.0
den2 = 1000.0
cstretch2 = 0.5
mat_spec2 = OrdinaryStateBasedSpecific([K2], [G2], [cstretch2], [den2])
# mat_spec2 = SkipSpecific([den2])

block4 = PeridynamicsMaterial(mat_gen2, mat_spec2; name="block 3")

Average family members: 47.475


OrdinaryStateBasedMaterial("block 3", 4:4, GeneralMaterial
type: [4]
horizon: 3.0
particle size: [1.0]
, OrdinaryStateBasedSpecific
bulk_modulus: [1.0]
shear_modulus: [1.0]
critical_stretch: [0.5]
density: [1000.0]
)

In [57]:
k = 1000.0
blocks = [block1, block2, block3, block4]

RM = [
        [LinearRepulsionModel(k, b1, distanceX=2, max_neighs=200) for b1 in blocks]..., 
        [LinearRepulsionModel(k, blocks[i], blocks[j], distanceX=2, max_neighs=200) for i in 2:4 for j in 1:i-1]...
    ]

env = Env(1, blocks, RM, [], 0.5)


bc = [
        MoveBC(env.type .== 4, [0.01, 0, 0]),
        FixBC(env.type .== 1),
        FixBC(env.type .== 2),
    ]

env = Env(1, blocks, RM, bc, 0.5)


nothing
#


In [58]:

velocity_verlet!([env], 5000, filewrite_freq=10, neigh_update_freq=10, file_prefix="half-cylinder", start_at=0)


Updating neighbors for collision..................
Average repulsive neighs: 0.0
Average repulsive neighs: 0.0
Average repulsive neighs: 23.331930693069307
Average repulsive neighs: 0.0
No repulsive neighs.
Average repulsive neighs: NaN
Average repulsive neighs: NaN
Average repulsive neighs: NaN
Average repulsive neighs: NaN
Average repulsive neighs: NaN
Done
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Momentum: 1[3200.0; 6.049333126437184e-15; -3.552713678800501e-15]

Writting data file................................
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Material force calculation passed. [Undeformed]
Done
0.02%
Material force calculation passed. [Undeformed]
Material force calculation 

LoadError: InterruptException:

In [48]:
using StaticArrays, Folds

function PeriDyn.force_density_T(y::Array{Float64,2}, mat::OrdinaryStateBasedMaterial)
    force = zeros(size(y))
    types = mat.general.type
    S = mat.general
    m = mat.general.weighted_volume
    intact = S.intact
    family = S.family
    PeriDyn.@timeit theta = dilatation(y, S, m) "dilation"
    N = size(S.x, 2)

    Threads.@threads for i in 1:N
        for k in 1:size(S.family,1)
            j = S.family[k,i]
            if !S.intact[k,i]
                S.intact[k,i] = false
            else
                X = PeriDyn.@_ij(j,i,S.x)
                Y = PeriDyn.@_ij(j,i,y)
                yij = PeriDyn.@_magnitude(Y)
                xij = PeriDyn.@_magnitude(X)
                
                extention = yij - xij
                wij = influence_function(X)
                wji = influence_function(-X)
                type1 = types[i] - mat.type.start + 1
                type2 = types[j] - mat.type.start + 1
                if (extention/xij) < mat.specific.critical_stretch[type1, type2]
                    K = mat.specific.bulk_modulus[type1, type2]
                    G = mat.specific.shear_modulus[type1, type2]
                    a_, b_ = wij/m[i], wji/m[j]
                    t =  (3*K-5*G) * (theta[i]*xij*a_ + theta[j]*xij*b_)  + 15*G*extention*(a_ + b_) 
                    t = t*S.volume[j] / yij
                    force[1,i] += t*Y[1]
                    force[2,i] += t*Y[2]
                    force[3,i] += t*Y[3]
                else
                    S.intact[k,i] = false
                end
            end
        end
    end

    return force
end

PeriDyn.@timeit force_density_T(1.1*block1.general.x, block1) ""

  0.000197 seconds (1.02 k allocations: 213.516 KiB)
Time taken for dilation: 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################
  0.257275 seconds (321.26 k allocations: 18.100 MiB, 41.51% compilation time)
Time taken for : 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################


3×320 Matrix{Float64}:
 0.305816  0.134867  -0.134867  -0.305816  …   0.134867  -0.134867  -0.305816
 0.305816  0.341934   0.341934   0.305816     -0.341934  -0.341934  -0.305816
 0.29763   0.330642   0.330642   0.29763      -0.330642  -0.330642  -0.29763

In [25]:
function PeriDyn.dilatation_theta(y::Array{Float64,2},S::GeneralMaterial)
    intact = S.intact
    family = S.family
    N = size(family, 2)

    function with_if_cal_theta_ij(i, j)
        X = PeriDyn.@_ij(j, i, S.x)
        Y = PeriDyn.@_ij(j, i, y)
        _X = PeriDyn.@_magnitude(X)
        _Y = PeriDyn.@_magnitude(Y)
        extention = _Y - _X
        return influence_function(X) * _X * extention * S.volume[j] * horizon_correction(X, S.particle_size, S.horizon)
    end

    inner_map(i, js) = mapreduce((j)-> with_if_cal_theta_ij(i,j), +, js)
    return Folds.map((i)->inner_map(i, family[intact[:, i], i]), 1:N)
    
end


S = block1.general
PeriDyn.@timeit dilatation(S.x*1.1, S, S.weighted_volume) ""
nothing

  0.091817 seconds (184.72 k allocations: 10.831 MiB, 88.24% compilation time)
Time taken for : 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################


In [174]:
function PeriDyn.dilatation_theta(y::Array{Float64,2},S::GeneralMaterial)
    intact = S.intact
    family = S.family
    N = size(family, 2)
    M = size(family, 1)
    theta = 0*S.volume
    Threads.@threads for i in 1:N
        for k in 1:M
            j = S.family[k,i]
            if (j>0) & (intact[k, i])
                X = PeriDyn.@_ij(j, i, S.x)
                Y = PeriDyn.@_ij(j, i, y)
                _X = PeriDyn.@_magnitude(X)
                _Y = PeriDyn.@_magnitude(Y)
                extention = _Y - _X
                theta[i] += influence_function(X)*_X*extention*S.volume[j] * horizon_correction(X, S.particle_size, S.horizon)
            end
        end
    end
    return theta
end


S = block1.general
PeriDyn.@timeit dilatation(S.x*1.1, S, S.weighted_volume) ""
nothing

  0.103823 seconds (60.03 k allocations: 7.990 MiB, 66.31% compilation time)
Time taken for : 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################


In [163]:
function PeriDyn.weighted_volume(x, volume, particle_size, family, horizon)
    N = size(family, 2)
    mask = family .> 0

    function cal_m_ij(i, j)
        X = [x[1,j]-x[1,i], x[2,j]-x[2,i], x[3,j]-x[3,i]]
        return influence_function(X)*_magnitude(X)^2*horizon_correction(X, particle_size, horizon)*volume[j]
    end
        
    inner_map(i, js) = mapreduce((j)-> cal_m_ij(i,j), +, js)
    return Folds.map((i)->inner_map(i, family[mask[:, i], i]), 1:N)
end


S = block1.general
PeriDyn.@timeit weighted_volume(S.x, S.volume, S.particle_size, S.family, S.horizon) ""
nothing

  4.320906 seconds (93.34 M allocations: 10.244 GiB, 40.22% gc time, 10.20% compilation time)
Time taken for : 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################


In [165]:
function PeriDyn.weighted_volume(x, volume, particle_size, family, horizon)
    m = 0*volume
    dr = zeros(3)
    for i in 1:size(x,2)
        for k in 1:size(family,1)
            j = family[k,i]
            if (j>0)
                dr[1] = x[1,j]-x[1,i]
                dr[2] = x[2,j]-x[2,i]
                dr[3] = x[3,j]-x[3,i]
                m[i] += influence_function(dr)*_magnitude(dr)^2*horizon_correction(dr, particle_size,horizon)*volume[j]
            end
        end
    end
    return m
end

S = block1.general
PeriDyn.@timeit weighted_volume(S.x, S.volume, S.particle_size, S.family, S.horizon) ""
nothing

  0.684199 seconds (41.46 k allocations: 10.047 MiB, 6.13% compilation time)
Time taken for : 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################


In [154]:
# @code_typed weighted_volume(S.x, S.volume, S.particle_size, S.family, S.horizon)

In [164]:
velocity_verlet!([env], 5000, filewrite_freq=10, neigh_update_freq=10, file_prefix="2blocks", start_at=0)


Updating neighbors for collision..................
Average repulsive neighs: 15.625
Done
Material force calculation passed. [Undeformed]
  0.752817 seconds (975.06 k allocations: 175.212 MiB, 4.31% gc time, 86.93% compilation time)
Time taken for Material force calculations (block 1): 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################
Material force calculation passed. [Undeformed]
  0.000137 seconds (70 allocations: 127.156 KiB)
Time taken for Material force calculations (block 2): 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################
  1.223970 seconds (1.80 M allocations: 167.084 MiB, 3.79% gc time, 92.36% compilation time)
Time taken for Short range repulsion calculations: 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#############################################################
Material force calculation passed. [Undefor

LoadError: InterruptException:

## END

In [69]:
function PeriDyn.td_norm_fn(y, theta, mat)
    S = mat.general
    types = mat.general.type
    intact = S.intact
    family = S.family
    x = S.x
    N = size(family, 2)
    M = size(family, 1)
    ARGS = map((i) -> (i, 1:M), 1:N)    
    m = mat.general.weighted_volume
    function with_if_cal_td_norm(i, k)
        if intact[k,i]
            j = family[k, i]
            type1 = types[i] - mat.type.start + 1
            type2 = types[j] - mat.type.start + 1
            G = mat.specific.shear_modulus[type1, type2]
            edp = mat.specific.edp[k, i]

            X = [x[1,j]-x[1,i],x[2,j]-x[2,i],x[3,j]-x[3,i]]
            Y = [y[1,j]-y[1,i],y[2,j]-y[2,i],y[3,j]-y[3,i]]

            _X = _magnitude(X)
            _Y = _magnitude(Y)

            e = _Y - _X

            ed = e - theta[i] * _X / 3
                        
            ee = ed - edp

            s = e/_X

            wij = influence_function(X)::Float64
            wji = influence_function(-X)::Float64
            type1 = types[i] - mat.type.start + 1
            type2 = types[j] - mat.type.start + 1
            if s < mat.specific.critical_stretch[type1, type2]
                K = mat.specific.bulk_modulus[type1, type2]
                G = mat.specific.shear_modulus[type1, type2]
                td = 15*G*(ee*wij/m[i]+ee*wji/m[j])
                return td
            else
                intact[k,i] = false
                return 0.0
            end            
        else
            return 0.0
        end
    end

    inner_map(i, inds) = PeriDyn.norm(map((j)-> with_if_cal_td_norm(i,j), inds))
    outer_map(ARGS) = map((x)->inner_map(x[1], x[2]), ARGS)
    return outer_map(ARGS)
end



In [122]:
function PeriDyn.force_density_T(y::Array{Float64,2}, mat::ElastoPlasticSolidMaterial)
    types = mat.general.type
    force = zeros(size(y))
    X = Float64[1.0,0.0,0.0]
    Y = Float64[1.0,0.0,0.0]
    S = mat.general
    m = mat.general.weighted_volume
    theta = dilatation(y, S, m)
    td_norm = PeriDyn.td_norm_fn(y, theta, mat)
    δ = S.horizon
    edp = mat.specific.edp
    N = size(S.x, 2)
        
    for i in 1:N
        for k in 1:size(S.family,1)
            j = S.family[k,i]::Int64
            if !S.intact[k,i] || i==j
                # force[1,i] += 0.0
                # force[2,i] += 0.0
                # force[3,i] += 0.0
                S.intact[k,i] = false
            else
                type1 = types[i] - mat.type.start + 1
                type2 = types[j] - mat.type.start + 1
                K = mat.specific.bulk_modulus[type1, type2]
                G = mat.specific.shear_modulus[type1, type2]
                σ = mat.specific.σy[type1, type2]
                X[1],X[2],X[3] = S.x[1,j]-S.x[1,i],S.x[2,j]-S.x[2,i],S.x[3,j]-S.x[3,i]
                Y[1],Y[2],Y[3] = y[1,j]-y[1,i],y[2,j]-y[2,i],y[3,j]-y[3,i]
                _Y, _X = _magnitude(Y), _magnitude(X)
                xij = _magnitude(X)::Float64
                wij = influence_function(X)::Float64
                wji = influence_function(-X)::Float64

                e = _Y - _X
                ed = e - theta[i] * _X / 3
                ee = ed - edp[i]

                td_trial = 15*G*(ee*wij/m[i]+ee*wji/m[j])

                if td_trial > 1.0e200
                    print(td_trial, " ", ee, " ", ed, " ", edp[i], " ", e/_X)
                    error("td trial")
                end
                            
                pyf, ψ = PeriDyn.plastic_yield_function(td_trial, σ, δ)

                if pyf < 0
                    td = td_trial
                else
                    if td_norm[i]==0.0
                        td = td_trial
                    else
                        α = 15*G/m[i]
                        td = sqrt(2*ψ) * td_trial / td_norm[i]
                        Δλ = 1/α*(td_norm[i]/sqrt(2*ψ) -1)
                        if abs(Δλ*(td)) > 1.0e10
                            print(edp[i], " ", Δλ, " ", sqrt(2*ψ), " ",  
                                            td_trial, " ", td_norm[i], " ",e/_X, " ", ed)
                            error("edpi")
                        end
                        edp[i] = edp[i] + Δλ*(td)
                    end
                    if isinf(td)
                        print(ψ, " ", td_trial, " ", ed, " ", td_norm[i])
                        error("td el")
                    end                                
                end
                            
                if isnan(edp[i])
                    error("edp")
                end
                            
                            
                if (e/xij) < mat.specific.critical_stretch[type1, type2]

                    t = (3*K)*(theta[i]*xij*wij/m[i]+theta[j]*xij*wji/m[j])  
                                
                    if isinf(t)
                        error("t i")
                    end
                    if isinf(td)
                        error("td")
                    end
                                
                    t = t + td
                    t = t*S.volume[i]/_magnitude(Y)

                    if isinf(t)
                        error("t")
                    end

                    force[1,i] += t*Y[1]
                    force[2,i] += t*Y[2]
                    force[3,i] += t*Y[3]
                                                    
                                
                    if sum(isnan.(force[:, i]))!=0
                        error("f i")
                    end

                else
                    # force[1,i] += 0.0
                    # force[2,i] += 0.0
                    # force[3,i] += 0.0
                    S.intact[k,i] = false
                end
            end
        end
    end
    if sum(isnan.(force))!=0
        error("force end")
    end
    return force
end

In [123]:
obj = Cuboid([0 6; 0 1; 0 1])
x1, v1, y1, vol1, type1 = create(obj, resolution=0.2, type=1)
Es = 69 # GPa
nu = 0.3333    # Poisson's Ratio
K = Es/3/(1-2nu)    # Bulk Modulus
G = Es/2/(1+nu)     # Shear Modulus
den = 2440.0        # Density in Kg m ^-3
horizon = 0.6
σ = 0.002*Es
s = 0.0002
mat_gen = GeneralMaterial(y1, v1, x1, vol1, type1, 0.3; max_neigh=200)
mat_spec = ElastoPlasticSolidSpecific([K] ,[G] ,[s] , [den],[σ])
block = PeridynamicsMaterial(mat_gen, mat_spec)

# fdt=force_density_T(1.002*y1,block)
BC1 = MoveBC(y1[1, :] .< 0.5,[-0.01, 0.0, 0.0])
BC2 = MoveBC(y1[1, :] .> 5.5, [0.01, 0.0, 0.0])
k = 0.000005
exp=4.0
RM1 = NonLinearRepulsionModel(exp,k, block; distanceX=3, max_neighs=200)
dt = 0.5
env = PeriDyn.Env(1, [block], [RM1], [BC1, BC2], dt)
udf = Int(1/dt)
Steps = 100*udf


velocity_verlet!([env], Steps, filewrite_freq=udf,neigh_update_freq=udf,file_prefix="vv",start_at=0)




Average family members: 13.88

Updating neighbors for collision..................Average repulsive neighs: 48.565333333333335
Done
Initial state saved.Momentum: 1[4.880000000000003; 0.0; 0.0]

Writting data file................................Done
0.5%

Writting data file................................Done

Updating neighbors for collision..................Average repulsive neighs: 48.74666666666667
1.0%
-1.7947877800850358e9 193622.02023510964 3.9485807317926054 2.6688422143347347e14 1.1368562763519337e10 0.0 1.7013651811515585e-18

LoadError: edpi